In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
avl_data = pd.read_csv('newdata_parsed.csv')

In [3]:
oneday = avl_data.query('TripDate=="2016-06-13"')

In [4]:
oneday = oneday.sort(columns=['vehicleID','RecordedAtTime'])

In [5]:
oneday.drop_duplicates(subset=['vehicleID','RecordedAtTime','Latitude','Longitude'],inplace=True)

In [6]:
oneday['RecordedAtTime_parsed'] = oneday.RecordedAtTime.apply(ttools.parseActualTime,tdate='2016-06-13')

That finishes the parsing of the vehicle ping times.
Next is to look at the time deltas between the Siri API responses.

In [7]:
responsetimes = oneday.ResponseTimeStamp.unique()

In [8]:
responsetimes.sort()

In [9]:
responsetimes = pd.Series(data=responsetimes,index=responsetimes)

In [10]:
responsetimes = responsetimes.apply(ttools.parseActualTime,tdate='2016-06-13')

In [11]:
responsetimediffs = pd.DataFrame(responsetimes.diff(),columns=['TimeSinceLastSiriResponse'])

In [12]:
responsetimediffs.head()

,TimeSinceLastSiriResponse
2016-06-13T02:10:40.022-04:00,NaT
2016-06-13T02:11:15.880-04:00,00:00:35
2016-06-13T02:11:51.152-04:00,00:00:36
2016-06-13T02:12:25.817-04:00,00:00:34
2016-06-13T02:13:00.099-04:00,00:00:35


That finished calculating the Siri response time differences

In [13]:
oneday = oneday.merge(responsetimediffs,how='left',left_on='ResponseTimeStamp',right_index=True)

In [14]:
oneday['TimeSinceLastPing'] = oneday.RecordedAtTime_parsed.diff()

In [15]:
oneday.head(15)

,vehicleID,Line,RecordedAtTime,Latitude,Longitude,Trip,TripDate,ResponseTimeStamp,RecordedAtTime_parsed,TimeSinceLastSiriResponse,TimeSinceLastPing
663836,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:05:40.000-04:00,40.786901,-73.950176,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:06:00.048-04:00,13:05:40,00:01:28,NaT
666177,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:07:14.000-04:00,40.785463,-73.951226,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:07:33.389-04:00,13:07:14,00:00:03,00:01:34
1156394,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:08:28.000-04:00,40.783989,-73.952304,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:08:53.334-04:00,13:08:28,00:00:22,00:01:14
3826081,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:09:32.000-04:00,40.780415,-73.954914,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:10:00.135-04:00,13:09:32,00:00:04,00:01:04
4435928,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:10:04.000-04:00,40.779581,-73.955523,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:10:09.167-04:00,13:10:04,00:00:09,00:00:32
189959,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:11:08.000-04:00,40.778855,-73.956053,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:11:26.766-04:00,13:11:08,00:01:17,00:01:04
2658845,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:11:39.000-04:00,40.778855,-73.956053,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:11:41.402-04:00,13:11:39,00:00:15,00:00:31
1580839,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:12:44.000-04:00,40.778855,-73.956053,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:13:00.129-04:00,13:12:44,00:01:19,00:01:05
1200980,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:14:21.000-04:00,40.776306,-73.957914,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:14:30.117-04:00,13:14:21,00:01:28,00:01:37
4440650,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:14:53.000-04:00,40.775373,-73.958594,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:15:24.481-04:00,13:14:53,00:00:54,00:00:32


In [16]:
# gap for any reasons
gaps = oneday.TimeSinceLastPing>ttools.datetime.timedelta(seconds=45)
gaps.head()

663836     False
666177      True
1156394     True
3826081     True
4435928    False
Name: TimeSinceLastPing, dtype: bool

In [17]:
# but we got a response from siri since then + 30 seconds
oneday.TimeSinceLastSiriResponse[gaps].head()

666177    00:00:03
1156394   00:00:22
3826081   00:00:04
189959    00:01:17
1580839   00:01:19
Name: TimeSinceLastSiriResponse, dtype: timedelta64[ns]

In [21]:
oneday[gaps].head(10)

,vehicleID,Line,RecordedAtTime,Latitude,Longitude,Trip,TripDate,ResponseTimeStamp,RecordedAtTime_parsed,TimeSinceLastSiriResponse,TimeSinceLastPing
666177,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:07:14.000-04:00,40.785463,-73.951226,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:07:33.389-04:00,13:07:14,00:00:03,00:01:34
1156394,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:08:28.000-04:00,40.783989,-73.952304,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:08:53.334-04:00,13:08:28,00:00:22,00:01:14
3826081,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:09:32.000-04:00,40.780415,-73.954914,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:10:00.135-04:00,13:09:32,00:00:04,00:01:04
189959,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:11:08.000-04:00,40.778855,-73.956053,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:11:26.766-04:00,13:11:08,00:01:17,00:01:04
1580839,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:12:44.000-04:00,40.778855,-73.956053,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:13:00.129-04:00,13:12:44,00:01:19,00:01:05
1200980,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:14:21.000-04:00,40.776306,-73.957914,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:14:30.117-04:00,13:14:21,00:01:28,00:01:37
692932,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:16:29.000-04:00,40.774441,-73.959274,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:17:00.096-04:00,13:16:29,00:01:19,00:01:04
695328,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:18:06.000-04:00,40.773643,-73.959854,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:18:30.130-04:00,13:18:06,00:01:22,00:01:05
5054877,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:19:10.000-04:00,40.773113,-73.960244,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:19:30.883-04:00,13:19:10,00:01:00,00:01:04
1205750,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:20:46.000-04:00,40.772552,-73.960653,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:20:57.115-04:00,13:20:46,00:01:11,00:01:03


In [31]:
# percent of pings that follow a gap
sum(gaps)/(len(gaps)*1.0)

0.51354414734344878

In [30]:
# these are definitely obscured by Siri gap
sum(oneday.TimeSinceLastPing[gaps] < oneday.TimeSinceLastSiriResponse[gaps])/(len(gaps)*1.0)

0.13833149697946398

In [46]:
# for which gaps was the last siri call at least 30 seconds after the last ping
# these are definitely a vehicle gap (at least, the first ping...)
sum(oneday.TimeSinceLastSiriResponse[gaps] < (oneday.TimeSinceLastPing[gaps] - ttools.datetime.timedelta(seconds=30)))/(len(gaps)*1.0)

0.15767868855916872